# Laborator 2b: Introducere în IDS/IPS

## Obiective
- Înțelegerea conceptelor IDS și IPS
- Instalarea și configurarea Snort
- Scrierea regulilor de detectare
- Analiza traficului cu Snort

## Cerințe
- Rulați acest notebook în Google Colab
- Snort va funcționa în mod offline (analiză PCAP)

## 1. Ce este IDS/IPS?

### IDS (Intrusion Detection System)
Un sistem care **monitorizează** traficul de rețea sau activitatea sistemului pentru a detecta activități malițioase sau încălcări ale politicilor de securitate.

**Analogie:** Camera de supraveghere - observă și alertează, dar nu intervine direct.

### IPS (Intrusion Prevention System)
Un sistem care nu doar detectează, dar și **blochează** activ traficul malițios.

**Analogie:** Bodyguard - observă, alertează și acționează pentru a opri amenințarea.

### Tipuri de IDS

| Tip | Descriere | Exemplu |
|-----|-----------|--------|
| **NIDS** (Network IDS) | Monitorizează traficul de rețea | Snort, Suricata |
| **HIDS** (Host IDS) | Monitorizează un singur host | OSSEC, Tripwire |

### Metode de detectare

1. **Signature-based**: Caută pattern-uri cunoscute (ca un antivirus)
2. **Anomaly-based**: Detectează abateri de la comportamentul normal
3. **Hybrid**: Combină ambele metode

## 2. Instalare Snort în Colab

**Snort** este cel mai popular IDS open-source, dezvoltat de Cisco.

> **Notă:** În Colab, Snort poate analiza doar fișiere PCAP (mod offline), nu trafic live.

In [ ]:
# Instalare Snort
!apt-get update -qq
!DEBIAN_FRONTEND=noninteractive apt-get install -y -qq snort > /dev/null 2>&1
print("Snort instalat cu succes!")

In [ ]:
# Verificare versiune
!snort --version 2>&1 | head -5

In [ ]:
# Structura directoarelor Snort
!echo "=== Configurare Snort ==="
!ls -la /etc/snort/ 2>/dev/null || echo "Director /etc/snort nu există"

!echo "\n=== Regulile Snort ==="
!ls /etc/snort/rules/ 2>/dev/null | head -10 || echo "Nu există reguli instalate"

## 3. Structura Regulilor Snort

O regulă Snort are două părți principale:

```
ACTION PROTOCOL SRC_IP SRC_PORT -> DST_IP DST_PORT (OPTIONS)
```

### Exemplu:
```
alert tcp any any -> 192.168.1.0/24 80 (msg:"HTTP Traffic"; sid:1000001;)
```

### Componente:

| Component | Descriere | Valori posibile |
|-----------|-----------|----------------|
| **ACTION** | Ce face când găsește match | alert, log, pass, drop |
| **PROTOCOL** | Protocolul de rețea | tcp, udp, icmp, ip |
| **SRC_IP** | IP sursă | any, IP specific, CIDR |
| **SRC_PORT** | Port sursă | any, număr, range |
| **->** | Direcția | -> (unidirecțional), <> (bidirecțional) |
| **DST_IP** | IP destinație | any, IP specific, CIDR |
| **DST_PORT** | Port destinație | any, număr, range |
| **OPTIONS** | Opțiuni adiționale | msg, content, sid, etc. |

### Opțiuni comune:
- `msg:"text"` - Mesajul alertei
- `sid:number` - ID unic al regulii (obligatoriu)
- `content:"text"` - Caută text în payload
- `nocase` - Case insensitive
- `rev:number` - Revizia regulii

## 4. Descărcare PCAP de Test

In [ ]:
# Descărcăm un PCAP public pentru testare
# Sursa: Wireshark Sample Captures
!wget -q "https://wiki.wireshark.org/uploads/__moin_import__/attachments/SampleCaptures/http.cap" -O http_traffic.pcap
!wget -q "https://wiki.wireshark.org/uploads/__moin_import__/attachments/SampleCaptures/telnet-raw.pcap" -O telnet_traffic.pcap

print("Fișiere PCAP descărcate:")
!ls -la *.pcap

In [ ]:
# Analiză rapidă cu tcpdump
!apt-get install -qq tcpdump > /dev/null 2>&1
print("=== Primele 10 pachete din http_traffic.pcap ===")
!tcpdump -r http_traffic.pcap -c 10 2>/dev/null

## 5. Crearea Regulilor Custom

Vom crea reguli pentru a detecta:
1. Trafic ICMP (ping)
2. Cereri HTTP
3. Conexiuni Telnet

In [ ]:
# Creare fișier cu reguli custom
custom_rules = """
# ============================================
# Reguli Custom pentru Laborator
# ============================================

# Regula 1: Detectare ICMP (Ping)
alert icmp any any -> any any (msg:"ICMP Ping Detected"; sid:1000001; rev:1;)

# Regula 2: Detectare cereri HTTP GET
alert tcp any any -> any 80 (msg:"HTTP GET Request"; content:"GET"; http_method; sid:1000002; rev:1;)

# Regula 3: Detectare conexiuni Telnet
alert tcp any any -> any 23 (msg:"Telnet Connection Attempt"; sid:1000003; rev:1;)

# Regula 4: Detectare HTTP cu "password" în request
alert tcp any any -> any 80 (msg:"Possible Password in HTTP"; content:"password"; nocase; sid:1000004; rev:1;)

# Regula 5: Detectare SSH
alert tcp any any -> any 22 (msg:"SSH Connection"; sid:1000005; rev:1;)

# Regula 6: Detectare DNS query
alert udp any any -> any 53 (msg:"DNS Query"; sid:1000006; rev:1;)
"""

with open('/tmp/local.rules', 'w') as f:
    f.write(custom_rules)

print("Reguli custom create în /tmp/local.rules")
print("\nConținut:")
!cat /tmp/local.rules

## 6. Configurare Snort Minimală

In [ ]:
# Configurare minimală pentru Snort
snort_conf = """
# Configurare minimală Snort pentru laborator

# Variabile de rețea
var HOME_NET any
var EXTERNAL_NET any

# Variabile pentru porturi
portvar HTTP_PORTS 80
portvar SSH_PORTS 22

# Calea către reguli
var RULE_PATH /tmp

# Include regulile noastre
include $RULE_PATH/local.rules
"""

with open('/tmp/snort.conf', 'w') as f:
    f.write(snort_conf)

print("Configurare Snort creată în /tmp/snort.conf")

## 7. Rulare Snort pe Fișiere PCAP

In [ ]:
# Rulare Snort pe traficul HTTP
print("=" * 60)
print("ANALIZĂ: http_traffic.pcap")
print("=" * 60)

!snort -r http_traffic.pcap -c /tmp/snort.conf -A console -q 2>/dev/null || echo "Eroare la rulare Snort"

In [ ]:
# Rulare Snort pe traficul Telnet
print("=" * 60)
print("ANALIZĂ: telnet_traffic.pcap")
print("=" * 60)

!snort -r telnet_traffic.pcap -c /tmp/snort.conf -A console -q 2>/dev/null || echo "Eroare la rulare Snort"

## 8. Interpretarea Alertelor

Formatul alertei Snort:
```
[**] [1:SID:REV] MESSAGE [**]
[Priority: X]
TIMESTAMP SRC_IP:PORT -> DST_IP:PORT
PROTOCOL TTL:X TOS:X ID:X ...
```

### Câmpuri importante:
- **SID** - ID-ul regulii care a declanșat alerta
- **MESSAGE** - Mesajul definit în regulă
- **Priority** - Prioritatea (1=high, 2=medium, 3=low)
- **SRC/DST** - Adresele și porturile implicate

## 9. Exercițiu: Detectare Port Scan

Vom crea o regulă pentru a detecta potențiale scanări de porturi.

In [ ]:
# Generăm un PCAP sintetic cu comportament de port scan
# folosind scapy

!pip install scapy -qq

from scapy.all import *
import random

packets = []

# Simulăm un SYN scan de la 192.168.1.100 către 192.168.1.1
attacker_ip = "192.168.1.100"
target_ip = "192.168.1.1"

# Scanăm 20 de porturi diferite
ports_to_scan = [21, 22, 23, 25, 53, 80, 110, 135, 139, 143, 
                 443, 445, 993, 995, 1433, 3306, 3389, 5432, 8080, 8443]

for port in ports_to_scan:
    # Pachet SYN
    pkt = IP(src=attacker_ip, dst=target_ip)/TCP(sport=random.randint(1024,65535), 
                                                   dport=port, 
                                                   flags='S')
    packets.append(pkt)

# Salvăm în PCAP
wrpcap('port_scan.pcap', packets)
print(f"Generat port_scan.pcap cu {len(packets)} pachete SYN")
print(f"Porturi scanate: {ports_to_scan}")

In [ ]:
# Adăugăm o regulă pentru detectarea SYN către multiple porturi
portscan_rule = """
# Detectare SYN flags (indicator de port scan)
alert tcp any any -> any any (msg:"Possible Port Scan - SYN packet"; flags:S; sid:1000010; rev:1;)

# Detectare SYN către porturi sensibile
alert tcp any any -> any 22 (msg:"SYN to SSH port"; flags:S; sid:1000011; rev:1;)
alert tcp any any -> any 23 (msg:"SYN to Telnet port"; flags:S; sid:1000012; rev:1;)
alert tcp any any -> any 3389 (msg:"SYN to RDP port"; flags:S; sid:1000013; rev:1;)
"""

# Adăugăm la regulile existente
with open('/tmp/local.rules', 'a') as f:
    f.write(portscan_rule)

print("Reguli pentru port scan adăugate!")

In [ ]:
# Rulare Snort pe PCAP-ul cu port scan
print("=" * 60)
print("ANALIZĂ: port_scan.pcap")
print("=" * 60)

!snort -r port_scan.pcap -c /tmp/snort.conf -A console -q 2>/dev/null | head -50

## 10. Comparație: IDS Signature-based vs ML-based

| Aspect | Signature-based (Snort) | ML-based |
|--------|------------------------|----------|
| **Detectare** | Pattern-uri cunoscute | Anomalii învățate |
| **Zero-day** | Nu detectează | Poate detecta |
| **False Positives** | Puține | Mai multe |
| **Actualizare** | Reguli noi manual | Re-antrenare model |
| **Interpretabilitate** | Ușor de înțeles | "Black box" |
| **Performanță** | Foarte rapidă | Depinde de model |

**Concluzie:** Sistemele moderne combină ambele abordări (Hybrid IDS).

## 11. Întrebări de Verificare

1. Care este diferența principală între IDS și IPS?
2. Ce tip de IDS este Snort: NIDS sau HIDS?
3. Ce semnifică `flags:S` într-o regulă Snort?
4. De ce un IDS signature-based nu poate detecta atacuri zero-day?
5. Care sunt avantajele și dezavantajele IPS față de IDS?
6. Cum ar arăta o regulă Snort pentru detectarea cererilor SSH?

## 12. Rezumat

În acest laborator am învățat:

1. **Conceptele IDS/IPS** - sisteme de detectare și prevenire a intruziunilor
2. **Tipuri de IDS** - NIDS (network) vs HIDS (host)
3. **Snort** - cel mai popular IDS open-source
4. **Structura regulilor** - ACTION PROTOCOL SRC -> DST (OPTIONS)
5. **Scrierea regulilor custom** - pentru detectarea diferitelor tipuri de trafic
6. **Analiza offline** - rularea Snort pe fișiere PCAP

### Următorul pas:
În laboratoarele următoare vom folosi Machine Learning pentru detecția intruziunilor, combinând abordarea signature-based cu cea anomaly-based.